# **log**

In [5]:
'''from google.colab import drive
drive.mount("/content/gdrive")'''

'from google.colab import drive\ndrive.mount("/content/gdrive")'

# **Prepare**

In [6]:
!pip install transformers
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
!pip install nltk
!pip install cuda-python
!pip install Keras-Preprocessing
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import math
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer

import warnings
warnings.filterwarnings('ignore')
import logging
logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)

import matplotlib.pyplot as plt
import tensorflow as tf
from transformers import BertTokenizer
from transformers import DistilBertTokenizer, RobertaTokenizer
from tensorflow.keras.optimizers import Adam, SGD
from transformers import TFXLNetModel, XLNetTokenizer
import nltk
import re
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve
from numpy import array
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from tensorflow.keras.layers import Embedding
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers import Concatenate
from torch.nn import BCEWithLogitsLoss
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, XLNetTokenizer, XLNetModel, XLNetLMHeadModel, XLNetConfig
from tqdm import tqdm, trange


Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic
ERROR: Could not find a version that satisfies t

In [7]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [8]:
train = pd.read_csv("train.scruples-anecdotes.csv" ,skip_blank_lines=True)

dev =pd.read_csv("dev-scruples-anecdotes.csv", skip_blank_lines=True)


test =pd.read_csv("test.scruples-anecdotes.csv", skip_blank_lines=True)



In [9]:
print(train['text'].isnull().sum())
print(dev['text'].isnull().sum())
print(test['text'].isnull().sum())

0
22
1
4


In [10]:
train = train.dropna()
dev = dev.dropna()
test = test.dropna()

In [11]:
print(train['text'].isnull().sum())
print(dev['text'].isnull().sum())
print(test['text'].isnull().sum())

0
0
0
0


In [12]:
train.shape

(26196, 10)

In [13]:
train

,id,post_id,action,title,text,post_type,label_scores,label,binarized_label_scores,binarized_label
0,yqk9ZSYu3Vd9A6sUBIGPLJ0mTCxcsv74,ar2iqw,"{'description': 'hiding my controller', 'prono...",AITA for hiding my controller?,"Backstory: So, I got an Xbox one for Christmas...",HISTORICAL,"{'AUTHOR': 0, 'OTHER': 22, 'EVERYBODY': 0, 'NO...",4,"{'RIGHT': 22, 'WRONG': 0}",RIGHT
1,xBCLN7LVJBGNAakxDVBy0fOrtpfTE577,aqtrol,"{'description': 'ratting out my supervisor', '...",AITA For Ratting Out My Supervisor?,I work with about six other people at might jo...,HISTORICAL,"{'AUTHOR': 0, 'OTHER': 5, 'EVERYBODY': 0, 'NOB...",4,"{'RIGHT': 5, 'WRONG': 0}",RIGHT
2,IdwLUU6ucUJfK8pL1Uv82dQpPVvgppO8,b01ajb,{'description': 'commenting that unfamiliar in...,AITA for commenting that unfamiliar indie band...,Context: There was an Instagram post about unp...,HISTORICAL,"{'AUTHOR': 3, 'OTHER': 4, 'EVERYBODY': 0, 'NOB...",4,"{'RIGHT': 5, 'WRONG': 3}",RIGHT
3,pbFtHLhjFcEF0MpceVPvk4oGmQeKof8D,a4m4zq,{'description': 'arguing for my friend to surr...,AITA for arguing for my Friend to Surrender in...,Me and my friends spent sometime organizing a ...,HISTORICAL,"{'AUTHOR': 5, 'OTHER': 0, 'EVERYBODY': 0, 'NOB...",0,"{'RIGHT': 0, 'WRONG': 5}",WRONG
4,IZzCnvsXj8vBAhArM0gxgALAwrd50yHG,apna7l,{'description': 'being mad at my wife for negl...,AITA for being mad at my wife for neglecting/ ...,A little background. I'm a far from rich guy w...,HISTORICAL,"{'AUTHOR': 0, 'OTHER': 5, 'EVERYBODY': 4, 'NOB...",4,"{'RIGHT': 5, 'WRONG': 4}",RIGHT
...,...,...,...,...,...,...,...,...,...,...
27761,MsxMg8eUAk7L2DuE1AT4IlwMzXZ90rUP,amjxh1,{'description': 'getting mad at my girlfriend ...,AITA for getting mad at my girlfriend for dism...,So a bit of background info. My girlfriend has...,HISTORICAL,"{'AUTHOR': 0, 'OTHER': 1, 'EVERYBODY': 1, 'NOB...",4,"{'RIGHT': 1, 'WRONG': 1}",RIGHT
27762,OYJdH0Swn7PxoOD3XfKdOtotoHLgMRvN,amuw37,"{'description': ""feeling shitty my best friend...",AITA for feeling shitty my best friend didn’t ...,Context:\n\nI’ve been best friends with this g...,HISTORICAL,"{'AUTHOR': 0, 'OTHER': 1, 'EVERYBODY': 0, 'NOB...",4,"{'RIGHT': 2, 'WRONG': 0}",RIGHT
27763,XOamt7UoD9nOsnQjNUWfsKx4jdGV708h,ajta6b,{'description': 'commenting on how my gf and h...,AITA for commenting on how my gf and her mum a...,So me (19M) and my gf’s (18F) relationship has...,HISTORICAL,"{'AUTHOR': 1, 'OTHER': 1, 'EVERYBODY': 0, 'NOB...",0,"{'RIGHT': 2, 'WRONG': 1}",RIGHT
27764,FJ2u1Tojs9VtXH52bb1iLBAdP6WYn3Yt,b0cdwc,{'description': 'missing out on family events ...,AITA for missing out on family events to work ...,"A little info, I’m an Early College student, f...",HISTORICAL,"{'AUTHOR': 1, 'OTHER': 1, 'EVERYBODY': 0, 'NOB...",3,"{'RIGHT': 3, 'WRONG': 1}",RIGHT


In [16]:
train_df = train[['text','binarized_label']]

In [17]:
dev_df = dev[['text','binarized_label']]

In [18]:
test_df =  test[['text','binarized_label']]

In [19]:
dev_df

,text,binarized_label
0,My boyfriend (well just call him M) and I are ...,RIGHT
2,\n\nMy really good friend is getting married....,RIGHT
3,"I've been very lonely for a long time, few fri...",RIGHT
4,Alt account because friends know my real one. ...,RIGHT
5,Pretty much was hanging out with two friends. ...,WRONG
...,...,...
2494,For reference we're both juniors at a T20 coll...,WRONG
2495,So this just happened and I'm a little taken a...,WRONG
2497,"So, I've been friends with someone for a few y...",RIGHT
2498,My first post!\n\nNo words were exchanged in t...,RIGHT


In [20]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

label_encoder = preprocessing.LabelEncoder()

train_df['num_binarized_label']= label_encoder.fit_transform(train_df['binarized_label'])
dev_df['num_binarized_label']= label_encoder.fit_transform(dev_df['binarized_label'])
test_df['num_binarized_label']= label_encoder.fit_transform(test_df['binarized_label'])


In [21]:
train_df

,text,binarized_label,num_binarized_label
0,"Backstory: So, I got an Xbox one for Christmas...",RIGHT,0
1,I work with about six other people at might jo...,RIGHT,0
2,Context: There was an Instagram post about unp...,RIGHT,0
3,Me and my friends spent sometime organizing a ...,WRONG,1
4,A little background. I'm a far from rich guy w...,RIGHT,0
...,...,...,...
27761,So a bit of background info. My girlfriend has...,RIGHT,0
27762,Context:\n\nI’ve been best friends with this g...,RIGHT,0
27763,So me (19M) and my gf’s (18F) relationship has...,RIGHT,0
27764,"A little info, I’m an Early College student, f...",RIGHT,0


In [22]:
nbl1 = train_df['num_binarized_label'] == 1
train_df_1 = pd.DataFrame(train_df[nbl1])
train_df_1

,text,binarized_label,num_binarized_label
3,Me and my friends spent sometime organizing a ...,WRONG,1
18,A girl I dated a few years ago and I have rema...,WRONG,1
22,TLDR at bottom.\n\nSo went to a nice dinner wi...,WRONG,1
27,"For some context, I live in a flatshare with t...",WRONG,1
30,I (24M) have been with my girlfriend(29F) for ...,WRONG,1
...,...,...,...
27742,It all started when I gave her the password to...,WRONG,1
27743,"Bit of a backstory, my boyfriend and I began d...",WRONG,1
27747,I work in a very small team at my company. So ...,WRONG,1
27751,"On Valentine’s Day, my GF and I had a great di...",WRONG,1


In [23]:
nbl0 = train_df['num_binarized_label'] == 0
train_df_new = pd.DataFrame(train_df[nbl0]) # dataframe that only contains '0' as num_binarized_label
#train_df_new
train_df_0 = train_df_new.iloc[:5738]
#train_df_0

In [24]:
balance_train_df = train_df_0.append(train_df_1, ignore_index=True)
#balance_train_df
train_df = balance_train_df.sample(frac = 1) # shuffle

In [25]:
train_df

,text,binarized_label,num_binarized_label
1088,As the title says I may or may not have caused...,RIGHT,0
8757,"Okay, so title says it all but now for the sto...",WRONG,1
11033,That's dramatic. \n\nIt needs a little back st...,WRONG,1
8455,Known each other for a year. To paint you a pi...,WRONG,1
1131,My GF and I have been together for about 6 yea...,RIGHT,0
...,...,...,...
7990,"The title is pretty self explanatory, but I gu...",WRONG,1
7514,(Throwaway account)\n\nMy good friend and I ha...,WRONG,1
11345,So I haven’t talked to my grandma since last F...,WRONG,1
2966,I’m crying as I type this. I have a lot of thi...,RIGHT,0


In [26]:
def clean_text(text):
    #Remove emojis and special chars
    clean=text
    #reg = re.compile('\\.+?(?=\B|$)')
    #clean = text.apply(lambda r: re.sub(reg, string=r, repl=''))
    #reg = re.compile('\x89Û_')
    #clean = clean.apply(lambda r: re.sub(reg, string=r, repl=' '))
    reg = re.compile('\&amp')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl='&'))
    reg = re.compile('\\n')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl=' '))
    
    #Remove hashtag symbol (#)
    #clean = clean.apply(lambda r: r.replace('#', ''))

    #Remove user names
    reg = re.compile('@[a-zA-Z0-9\_]+')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl='@'))

    #Remove URLs
    reg = re.compile('https?\S+(?=\s|$)')
    clean = clean.apply(lambda r: re.sub(reg, string=r, repl='www'))

    #Lowercase
    #clean = clean.apply(lambda r: r.lower())
    return clean

In [27]:
train_df = train_df.replace(np.nan, '', regex=True)

dev_df = dev_df.replace(np.nan, '', regex=True)

test_df = test_df.replace(np.nan, '', regex=True)


In [28]:
train_df['clean'] = clean_text(train_df['text'])
dev_df['clean'] = clean_text(dev_df['text'])
test_df['clean'] = clean_text(test_df['text'])

In [29]:
train_df

,text,binarized_label,num_binarized_label,clean
1088,As the title says I may or may not have caused...,RIGHT,0,As the title says I may or may not have caused...
8757,"Okay, so title says it all but now for the sto...",WRONG,1,"Okay, so title says it all but now for the sto..."
11033,That's dramatic. \n\nIt needs a little back st...,WRONG,1,That's dramatic. It needs a little back stor...
8455,Known each other for a year. To paint you a pi...,WRONG,1,Known each other for a year. To paint you a pi...
1131,My GF and I have been together for about 6 yea...,RIGHT,0,My GF and I have been together for about 6 yea...
...,...,...,...,...
7990,"The title is pretty self explanatory, but I gu...",WRONG,1,"The title is pretty self explanatory, but I gu..."
7514,(Throwaway account)\n\nMy good friend and I ha...,WRONG,1,(Throwaway account) My good friend and I have...
11345,So I haven’t talked to my grandma since last F...,WRONG,1,So I haven’t talked to my grandma since last F...
2966,I’m crying as I type this. I have a lot of thi...,RIGHT,0,I’m crying as I type this. I have a lot of thi...


In [30]:
test_df['clean'][0]

"   Throwaway for obvious reasons.     I dropped out of uni 2 years ago because I was planning on moving to a different country and going to Uni there. However, my parents didn't want to help me in any way and I couldn't afford it myself. They've always supported my younger sister though, bought her an apartment and they're even paying her bills, even though she earns enough money to do that herself. So atm I'm stuck at home with them, which sucks as I am 25 yo and I feel like I'm wasting my life completely.     Both my parents were extremely abusive, especially my dad. As a result, I have social anxiety, trust issues, panic attacks etc. I can deal with everything as I got used to it anyway, but lately things have gotten worse.     A few years ago my dad was diagnosed with type II diabetes, and he retired. He's been staying at home ever since and he's slowly but surely becoming a vegetable. He had a stroke which left him unable to speak/walk properly. I'm no doctor, but I found his hos

# **Data preprocessing**

In [31]:
train_df["lower"] = train_df["clean"].apply(lambda x: x.lower())

dev_df["lower"] = dev_df["clean"].apply(lambda x: x.lower())

test_df["lower"] = test_df["clean"].apply(lambda x: x.lower())
display(train_df.head())

,text,binarized_label,num_binarized_label,clean,lower
1088,As the title says I may or may not have caused...,RIGHT,0,As the title says I may or may not have caused...,as the title says i may or may not have caused...
8757,"Okay, so title says it all but now for the sto...",WRONG,1,"Okay, so title says it all but now for the sto...","okay, so title says it all but now for the sto..."
11033,That's dramatic. \n\nIt needs a little back st...,WRONG,1,That's dramatic. It needs a little back stor...,that's dramatic. it needs a little back stor...
8455,Known each other for a year. To paint you a pi...,WRONG,1,Known each other for a year. To paint you a pi...,known each other for a year. to paint you a pi...
1131,My GF and I have been together for about 6 yea...,RIGHT,0,My GF and I have been together for about 6 yea...,my gf and i have been together for about 6 yea...


In [32]:
!pip install contractions
import contractions
train_df["lower"] = train_df["lower"].apply(lambda x: contractions.fix(x))
dev_df["lower"] = dev_df["lower"].apply(lambda x: contractions.fix(x))
test_df["lower"] = test_df["lower"].apply(lambda x: contractions.fix(x))

Looking in links: /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/avx2, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/gentoo/generic, /cvmfs/soft.computecanada.ca/custom/python/wheelhouse/generic


In [33]:
def remove_non_ascii(text):
    """
        Remove non-ASCII characters 
    """
    return re.sub(r'[^\x00-\x7f]',r'', text) # or ''.join([x for x in text if x in string.printable]) 

# remove non-ascii characters from the text
train_df["lower"] = train_df["lower"].apply(lambda x: remove_non_ascii(x))
dev_df["lower"] = dev_df["lower"].apply(lambda x: remove_non_ascii(x))
test_df["lower"] = test_df["lower"].apply(lambda x: remove_non_ascii(x))

In [34]:
def remove_special_characters(text):
    """
        Remove special special characters, including symbols, emojis, and other graphic characters
    """
    emoji_pattern = re.compile(
        '['
        u'\U0001F600-\U0001F64F'  # emoticons
        u'\U0001F300-\U0001F5FF'  # symbols & pictographs
        u'\U0001F680-\U0001F6FF'  # transport & map symbols
        u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
        u'\U00002702-\U000027B0'
        u'\U000024C2-\U0001F251'
        ']+',
        flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


# remove non-ascii characters from the text
train_df["lower"] = train_df["lower"].apply(lambda x: remove_special_characters(x))
dev_df["lower"] = dev_df["lower"].apply(lambda x: remove_special_characters(x))
test_df["lower"] = test_df["lower"].apply(lambda x: remove_special_characters(x))

In [35]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/zhe8nov1/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [36]:
# Tokenizing the tweet base texts.
from nltk.tokenize import word_tokenize
train_df['tokenized'] = train_df['lower'].apply(word_tokenize)
dev_df['tokenized'] = dev_df['lower'].apply(word_tokenize)
test_df['tokenized'] = test_df['lower'].apply(word_tokenize)

In [37]:
from nltk.stem import SnowballStemmer

def snowball_stemmer(text):
    """
        Stem words in list of tokenized words with SnowballStemmer
    """
    stemmer = nltk.SnowballStemmer("english")
    stems = [stemmer.stem(i) for i in text]
    return stems


In [38]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/zhe8nov1/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/zhe8nov1/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [39]:
from nltk.stem import WordNetLemmatizer

def lemmatize_word(text):
    """
        Lemmatize the tokenized words
    """

    lemmatizer = WordNetLemmatizer()
    lemma = [lemmatizer.lemmatize(word, tag) for word, tag in text]
    return lemma



lemmatizer = WordNetLemmatizer()

train_df['lemmatize_word_wo_pos'] = train_df['tokenized'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

dev_df['lemmatize_word_wo_pos'] = dev_df['tokenized'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

test_df['lemmatize_word_wo_pos'] = test_df['tokenized'].apply(lambda x: [lemmatizer.lemmatize(word) for word in x])

In [40]:
connect1 = []
for i in train_df['lemmatize_word_wo_pos']:
  connect1.append(' '.join(i))
train_df['clean1'] = connect1


connect2 = []
for i in dev_df['lemmatize_word_wo_pos']:
  connect2.append(' '.join(i))
dev_df['clean1'] = connect2


connect3 = []
for i in test_df['lemmatize_word_wo_pos']:
  connect3.append(' '.join(i))
test_df['clean1'] = connect3


In [41]:
dev_df['clean'][0]

'My boyfriend (well just call him M) and I are creators on Instagram (he is also on TikTok). I\'m a digital and traditional artist and in one of my classes we create t shirts and things (socks, lanyards, water bottles, stickers, you name it). I\'m a big fan of putting my art on t shirts so I can sell them.  M wanted to make some posts for TikTok and wanted to use the shirt I made today. My one personal rule is that if the shirt isn\'t being given to someone else, then I wear it first (this is because I know I wont get it back. Ive made one other shirt that I\'ve worn, and he wore it for a week and hasn\'t given it back.) So I told him no, because I haven\'t worn the shirt yet.   M proceeded to complain about not having content to post, and how his IG post today didn\'t get any engagement, saying "of course you dont know, because you never look."  Am I the asshole for wanting to wear the shirt I made just once?'

In [42]:
dev_df

,text,binarized_label,num_binarized_label,clean,lower,tokenized,lemmatize_word_wo_pos,clean1
0,My boyfriend (well just call him M) and I are ...,RIGHT,0,My boyfriend (well just call him M) and I are ...,my boyfriend (well just call him m) and i are ...,"[my, boyfriend, (, well, just, call, him, m, )...","[my, boyfriend, (, well, just, call, him, m, )...",my boyfriend ( well just call him m ) and i ar...
2,\n\nMy really good friend is getting married....,RIGHT,0,My really good friend is getting married. Y...,my really good friend is getting married. y...,"[my, really, good, friend, is, getting, marrie...","[my, really, good, friend, is, getting, marrie...",my really good friend is getting married . yay...
3,"I've been very lonely for a long time, few fri...",RIGHT,0,"I've been very lonely for a long time, few fri...","i have been very lonely for a long time, few f...","[i, have, been, very, lonely, for, a, long, ti...","[i, have, been, very, lonely, for, a, long, ti...","i have been very lonely for a long time , few ..."
4,Alt account because friends know my real one. ...,RIGHT,0,Alt account because friends know my real one. ...,alt account because friends know my real one. ...,"[alt, account, because, friends, know, my, rea...","[alt, account, because, friend, know, my, real...",alt account because friend know my real one . ...
5,Pretty much was hanging out with two friends. ...,WRONG,1,Pretty much was hanging out with two friends. ...,pretty much was hanging out with two friends. ...,"[pretty, much, was, hanging, out, with, two, f...","[pretty, much, wa, hanging, out, with, two, fr...",pretty much wa hanging out with two friend . i...
...,...,...,...,...,...,...,...,...
2494,For reference we're both juniors at a T20 coll...,WRONG,1,For reference we're both juniors at a T20 coll...,for reference we are both juniors at a t20 col...,"[for, reference, we, are, both, juniors, at, a...","[for, reference, we, are, both, junior, at, a,...",for reference we are both junior at a t20 coll...
2495,So this just happened and I'm a little taken a...,WRONG,1,So this just happened and I'm a little taken a...,so this just happened and i am a little taken ...,"[so, this, just, happened, and, i, am, a, litt...","[so, this, just, happened, and, i, am, a, litt...",so this just happened and i am a little taken ...
2497,"So, I've been friends with someone for a few y...",RIGHT,0,"So, I've been friends with someone for a few y...","so, i have been friends with someone for a few...","[so, ,, i, have, been, friends, with, someone,...","[so, ,, i, have, been, friend, with, someone, ...","so , i have been friend with someone for a few..."
2498,My first post!\n\nNo words were exchanged in t...,RIGHT,0,My first post! No words were exchanged in thi...,my first post! no words were exchanged in thi...,"[my, first, post, !, no, words, were, exchange...","[my, first, post, !, no, word, were, exchanged...",my first post ! no word were exchanged in this...


In [43]:
train_df

,text,binarized_label,num_binarized_label,clean,lower,tokenized,lemmatize_word_wo_pos,clean1
1088,As the title says I may or may not have caused...,RIGHT,0,As the title says I may or may not have caused...,as the title says i may or may not have caused...,"[as, the, title, says, i, may, or, may, not, h...","[a, the, title, say, i, may, or, may, not, hav...",a the title say i may or may not have caused o...
8757,"Okay, so title says it all but now for the sto...",WRONG,1,"Okay, so title says it all but now for the sto...","okay, so title says it all but now for the sto...","[okay, ,, so, title, says, it, all, but, now, ...","[okay, ,, so, title, say, it, all, but, now, f...","okay , so title say it all but now for the sto..."
11033,That's dramatic. \n\nIt needs a little back st...,WRONG,1,That's dramatic. It needs a little back stor...,that is dramatic. it needs a little back sto...,"[that, is, dramatic, ., it, needs, a, little, ...","[that, is, dramatic, ., it, need, a, little, b...",that is dramatic . it need a little back story...
8455,Known each other for a year. To paint you a pi...,WRONG,1,Known each other for a year. To paint you a pi...,known each other for a year. to paint you a pi...,"[known, each, other, for, a, year, ., to, pain...","[known, each, other, for, a, year, ., to, pain...",known each other for a year . to paint you a p...
1131,My GF and I have been together for about 6 yea...,RIGHT,0,My GF and I have been together for about 6 yea...,my gf and i have been together for about 6 yea...,"[my, gf, and, i, have, been, together, for, ab...","[my, gf, and, i, have, been, together, for, ab...",my gf and i have been together for about 6 yea...
...,...,...,...,...,...,...,...,...
7990,"The title is pretty self explanatory, but I gu...",WRONG,1,"The title is pretty self explanatory, but I gu...","the title is pretty self explanatory, but i gu...","[the, title, is, pretty, self, explanatory, ,,...","[the, title, is, pretty, self, explanatory, ,,...","the title is pretty self explanatory , but i g..."
7514,(Throwaway account)\n\nMy good friend and I ha...,WRONG,1,(Throwaway account) My good friend and I have...,(throwaway account) my good friend and i have...,"[(, throwaway, account, ), my, good, friend, a...","[(, throwaway, account, ), my, good, friend, a...",( throwaway account ) my good friend and i hav...
11345,So I haven’t talked to my grandma since last F...,WRONG,1,So I haven’t talked to my grandma since last F...,so i have not talked to my grandma since last ...,"[so, i, have, not, talked, to, my, grandma, si...","[so, i, have, not, talked, to, my, grandma, si...",so i have not talked to my grandma since last ...
2966,I’m crying as I type this. I have a lot of thi...,RIGHT,0,I’m crying as I type this. I have a lot of thi...,i am crying as i type this. i have a lot of th...,"[i, am, crying, as, i, type, this, ., i, have,...","[i, am, cry, a, i, type, this, ., i, have, a, ...",i am cry a i type this . i have a lot of thing...


# **Testing story saperation**

In [44]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

data = dev_df['clean'][0]
print('\n-----\n'.join(tokenizer.tokenize(data)))

My boyfriend (well just call him M) and I are creators on Instagram (he is also on TikTok).
-----
I'm a digital and traditional artist and in one of my classes we create t shirts and things (socks, lanyards, water bottles, stickers, you name it).
-----
I'm a big fan of putting my art on t shirts so I can sell them.
-----
M wanted to make some posts for TikTok and wanted to use the shirt I made today.
-----
My one personal rule is that if the shirt isn't being given to someone else, then I wear it first (this is because I know I wont get it back.
-----
Ive made one other shirt that I've worn, and he wore it for a week and hasn't given it back.)
-----
So I told him no, because I haven't worn the shirt yet.
-----
M proceeded to complain about not having content to post, and how his IG post today didn't get any engagement, saying "of course you dont know, because you never look."
-----
Am I the asshole for wanting to wear the shirt I made just once?


In [45]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

data = dev_df['clean1'][0]
x=tokenizer.tokenize(data)
for i in x:
  print(i)
print(len(x))

my boyfriend ( well just call him m ) and i are creator on instagram ( he is also on tiktok ) .
i am a digital and traditional artist and in one of my class we create t shirt and thing ( sock , lanyard , water bottle , sticker , you name it ) .
i am a big fan of putting my art on t shirt so i can sell them .
m wanted to make some post for tiktok and wanted to use the shirt i made today .
my one personal rule is that if the shirt is not being given to someone else , then i wear it first ( this is because i know i will not get it back .
i have made one other shirt that i have worn , and he wore it for a week and ha not given it back . )
so i told him no , because i have not worn the shirt yet .
m proceeded to complain about not having content to post , and how his ig post today did not get any engagement , saying `` of course you do not know , because you never look . ''
am i the asshole for wanting to wear the shirt i made just once ?
9


In [46]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

train_sum = 0
num_of_sentences = []
# loop all the dataset, get text data and its label
train_line = []
train_line_label = []
for index, row in train_df.iterrows():
  sentence_count = 0 # for every new story, set the sentence counter to 0
  data = train_df['clean1'][index]
  label =  train_df['num_binarized_label'][index]
  saparated_text = tokenizer.tokenize(data)
  train_sum += len(saparated_text)
  for each_sentence in saparated_text: # loop through each saparated sentences.
    sentence_count += 1
    train_line.append(each_sentence) # add EACH SENTENCES to the list(line)
    train_line_label.append(label) # add its label to the list (line_label)
  num_of_sentences.append(sentence_count)
train_sentences = pd.DataFrame(train_line,columns=['line'])
train_sentences['line_label'] = train_line_label
if len(train_line) == train_sum:
  print('Training set has same len')
  train_df['num_of_sentences'] = num_of_sentences



dev_sum = 0
num_of_sentences = []
# loop all the dataset, get text data and its label
dev_line = []
dev_line_label = []
for index, row in dev_df.iterrows():
  sentence_count = 0 # for every new story, set the sentence counter to 0
  data = dev_df['clean1'][index]
  label =  dev_df['num_binarized_label'][index]
  saparated_text = tokenizer.tokenize(data)
  dev_sum += len(saparated_text)
  for each_sentence in saparated_text: # loop through each saparated sentences.
    sentence_count += 1
    dev_line.append(each_sentence) # add it to the list(line)
    dev_line_label.append(label) # add its label to the list (line_label)
  num_of_sentences.append(sentence_count)
dev_sentences = pd.DataFrame(dev_line,columns=['line'])
dev_sentences['line_label'] = dev_line_label
if len(dev_line) == dev_sum:
  print('DEV set has same len')
  dev_df['num_of_sentences'] = num_of_sentences



test_sum = 0
num_of_sentences = []
 # loop all the dataset, get text data and its label
test_line = []
test_line_label = []
for index, row in test_df.iterrows():
  sentence_count = 0 
  data = test_df['clean1'][index]
  label =  test_df['num_binarized_label'][index]
  saparated_text = tokenizer.tokenize(data)
  test_sum += len(saparated_text)
  for each_sentence in saparated_text: # loop through each saparated sentences.
    sentence_count += 1
    test_line.append(each_sentence) # add it to the list(line)
    test_line_label.append(label) # add its label to the list (line_label)
  num_of_sentences.append(sentence_count)
test_sentences = pd.DataFrame(test_line,columns=['line'])
test_sentences['line_label'] = test_line_label
if len(test_line) == test_sum:
  print('Test set has same len')
  test_df['num_of_sentences'] = num_of_sentences

Training set has same len
DEV set has same len
Test set has same len


In [47]:
train_df

,text,binarized_label,num_binarized_label,clean,lower,tokenized,lemmatize_word_wo_pos,clean1,num_of_sentences
1088,As the title says I may or may not have caused...,RIGHT,0,As the title says I may or may not have caused...,as the title says i may or may not have caused...,"[as, the, title, says, i, may, or, may, not, h...","[a, the, title, say, i, may, or, may, not, hav...",a the title say i may or may not have caused o...,11
8757,"Okay, so title says it all but now for the sto...",WRONG,1,"Okay, so title says it all but now for the sto...","okay, so title says it all but now for the sto...","[okay, ,, so, title, says, it, all, but, now, ...","[okay, ,, so, title, say, it, all, but, now, f...","okay , so title say it all but now for the sto...",11
11033,That's dramatic. \n\nIt needs a little back st...,WRONG,1,That's dramatic. It needs a little back stor...,that is dramatic. it needs a little back sto...,"[that, is, dramatic, ., it, needs, a, little, ...","[that, is, dramatic, ., it, need, a, little, b...",that is dramatic . it need a little back story...,30
8455,Known each other for a year. To paint you a pi...,WRONG,1,Known each other for a year. To paint you a pi...,known each other for a year. to paint you a pi...,"[known, each, other, for, a, year, ., to, pain...","[known, each, other, for, a, year, ., to, pain...",known each other for a year . to paint you a p...,8
1131,My GF and I have been together for about 6 yea...,RIGHT,0,My GF and I have been together for about 6 yea...,my gf and i have been together for about 6 yea...,"[my, gf, and, i, have, been, together, for, ab...","[my, gf, and, i, have, been, together, for, ab...",my gf and i have been together for about 6 yea...,8
...,...,...,...,...,...,...,...,...,...
7990,"The title is pretty self explanatory, but I gu...",WRONG,1,"The title is pretty self explanatory, but I gu...","the title is pretty self explanatory, but i gu...","[the, title, is, pretty, self, explanatory, ,,...","[the, title, is, pretty, self, explanatory, ,,...","the title is pretty self explanatory , but i g...",10
7514,(Throwaway account)\n\nMy good friend and I ha...,WRONG,1,(Throwaway account) My good friend and I have...,(throwaway account) my good friend and i have...,"[(, throwaway, account, ), my, good, friend, a...","[(, throwaway, account, ), my, good, friend, a...",( throwaway account ) my good friend and i hav...,7
11345,So I haven’t talked to my grandma since last F...,WRONG,1,So I haven’t talked to my grandma since last F...,so i have not talked to my grandma since last ...,"[so, i, have, not, talked, to, my, grandma, si...","[so, i, have, not, talked, to, my, grandma, si...",so i have not talked to my grandma since last ...,8
2966,I’m crying as I type this. I have a lot of thi...,RIGHT,0,I’m crying as I type this. I have a lot of thi...,i am crying as i type this. i have a lot of th...,"[i, am, crying, as, i, type, this, ., i, have,...","[i, am, cry, a, i, type, this, ., i, have, a, ...",i am cry a i type this . i have a lot of thing...,27


In [48]:
dev_df

,text,binarized_label,num_binarized_label,clean,lower,tokenized,lemmatize_word_wo_pos,clean1,num_of_sentences
0,My boyfriend (well just call him M) and I are ...,RIGHT,0,My boyfriend (well just call him M) and I are ...,my boyfriend (well just call him m) and i are ...,"[my, boyfriend, (, well, just, call, him, m, )...","[my, boyfriend, (, well, just, call, him, m, )...",my boyfriend ( well just call him m ) and i ar...,9
2,\n\nMy really good friend is getting married....,RIGHT,0,My really good friend is getting married. Y...,my really good friend is getting married. y...,"[my, really, good, friend, is, getting, marrie...","[my, really, good, friend, is, getting, marrie...",my really good friend is getting married . yay...,25
3,"I've been very lonely for a long time, few fri...",RIGHT,0,"I've been very lonely for a long time, few fri...","i have been very lonely for a long time, few f...","[i, have, been, very, lonely, for, a, long, ti...","[i, have, been, very, lonely, for, a, long, ti...","i have been very lonely for a long time , few ...",6
4,Alt account because friends know my real one. ...,RIGHT,0,Alt account because friends know my real one. ...,alt account because friends know my real one. ...,"[alt, account, because, friends, know, my, rea...","[alt, account, because, friend, know, my, real...",alt account because friend know my real one . ...,18
5,Pretty much was hanging out with two friends. ...,WRONG,1,Pretty much was hanging out with two friends. ...,pretty much was hanging out with two friends. ...,"[pretty, much, was, hanging, out, with, two, f...","[pretty, much, wa, hanging, out, with, two, fr...",pretty much wa hanging out with two friend . i...,8
...,...,...,...,...,...,...,...,...,...
2494,For reference we're both juniors at a T20 coll...,WRONG,1,For reference we're both juniors at a T20 coll...,for reference we are both juniors at a t20 col...,"[for, reference, we, are, both, juniors, at, a...","[for, reference, we, are, both, junior, at, a,...",for reference we are both junior at a t20 coll...,25
2495,So this just happened and I'm a little taken a...,WRONG,1,So this just happened and I'm a little taken a...,so this just happened and i am a little taken ...,"[so, this, just, happened, and, i, am, a, litt...","[so, this, just, happened, and, i, am, a, litt...",so this just happened and i am a little taken ...,13
2497,"So, I've been friends with someone for a few y...",RIGHT,0,"So, I've been friends with someone for a few y...","so, i have been friends with someone for a few...","[so, ,, i, have, been, friends, with, someone,...","[so, ,, i, have, been, friend, with, someone, ...","so , i have been friend with someone for a few...",37
2498,My first post!\n\nNo words were exchanged in t...,RIGHT,0,My first post! No words were exchanged in thi...,my first post! no words were exchanged in thi...,"[my, first, post, !, no, words, were, exchange...","[my, first, post, !, no, word, were, exchanged...",my first post ! no word were exchanged in this...,25


In [72]:
test_df

,text,binarized_label,num_binarized_label,clean,lower,tokenized,lemmatize_word_wo_pos,clean1,num_of_sentences
0,Throwaway for obvious reasons.\n\n I drop...,WRONG,1,Throwaway for obvious reasons. I droppe...,throwaway for obvious reasons. i droppe...,"[throwaway, for, obvious, reasons, ., i, dropp...","[throwaway, for, obvious, reason, ., i, droppe...",throwaway for obvious reason . i dropped out o...,33
1,"For background, my mom's cousin is staying wit...",RIGHT,0,"For background, my mom's cousin is staying wit...","for background, my mom's cousin is staying wit...","[for, background, ,, my, mom, 's, cousin, is, ...","[for, background, ,, my, mom, 's, cousin, is, ...","for background , my mom 's cousin is staying w...",14
3,Background: I can't give much detail because o...,WRONG,1,Background: I can't give much detail because o...,background: i cannot give much detail because ...,"[background, :, i, can, not, give, much, detai...","[background, :, i, can, not, give, much, detai...",background : i can not give much detail becaus...,26
4,My girlfriend volunteers at an animal shelter ...,RIGHT,0,My girlfriend volunteers at an animal shelter ...,my girlfriend volunteers at an animal shelter ...,"[my, girlfriend, volunteers, at, an, animal, s...","[my, girlfriend, volunteer, at, an, animal, sh...",my girlfriend volunteer at an animal shelter a...,13
5,This has been on my mind for a while now and I...,RIGHT,0,This has been on my mind for a while now and I...,this has been on my mind for a while now and i...,"[this, has, been, on, my, mind, for, a, while,...","[this, ha, been, on, my, mind, for, a, while, ...",this ha been on my mind for a while now and i ...,20
...,...,...,...,...,...,...,...,...,...
2493,I want to preface this by saying that this fri...,RIGHT,0,I want to preface this by saying that this fri...,i want to preface this by saying that this fri...,"[i, want, to, preface, this, by, saying, that,...","[i, want, to, preface, this, by, saying, that,...",i want to preface this by saying that this fri...,11
2494,"Some backstory here, I’m having a party tonigh...",RIGHT,0,"Some backstory here, I’m having a party tonigh...","some backstory here, i am having a party tonig...","[some, backstory, here, ,, i, am, having, a, p...","[some, backstory, here, ,, i, am, having, a, p...","some backstory here , i am having a party toni...",16
2495,Okay this is a long 2 part one so strap in and...,RIGHT,0,Okay this is a long 2 part one so strap in and...,okay this is a long 2 part one so strap in and...,"[okay, this, is, a, long, 2, part, one, so, st...","[okay, this, is, a, long, 2, part, one, so, st...",okay this is a long 2 part one so strap in and...,30
2497,"First of all, I should precise that I am a str...",WRONG,1,"First of all, I should precise that I am a str...","first of all, i should precise that i am a str...","[first, of, all, ,, i, should, precise, that, ...","[first, of, all, ,, i, should, precise, that, ...","first of all , i should precise that i am a st...",33


In [49]:
train_sentences

,line,line_label
0,a the title say i may or may not have caused o...,0
1,i ( m ) have been friend with c ( f ) for arou...,0
2,t ha not had the greatest track record when it...,0
3,"me being a friend of both party , ( i am no lo...",0
4,"but with me being c best male friend , i decid...",0
...,...,...
198620,my mind wandered over to her dirty clothes ham...,1
198621,"i grabbed them , and they still reeked - it tu...",1
198622,i jerked off and finished really quickly ( but...,1
198623,am i an asshole for doing this ?,1


In [50]:
dev_sentences

,line,line_label
0,my boyfriend ( well just call him m ) and i ar...,0
1,i am a digital and traditional artist and in o...,0
2,i am a big fan of putting my art on t shirt so...,0
3,m wanted to make some post for tiktok and want...,0
4,my one personal rule is that if the shirt is n...,0
...,...,...
40891,after this incident we ( my suite mate and i )...,1
40892,he had broken my trust in a way he can not reg...,1
40893,we explained to her that it looked creepy and ...,1
40894,now my roommate seems annoyed that i do not wa...,1


In [51]:
test_sentences

,line,line_label
0,throwaway for obvious reason .,1
1,i dropped out of uni 2 year ago because i wa p...,1
2,"however , my parent did not want to help me in...",1
3,they have always supported my younger sister t...,1
4,"so atm i am stuck at home with them , which su...",1
...,...,...
41130,"the only class we share are math and form , wh...",0
41131,i spend the whole of lunch with her on wednesd...,0
41132,but i just hate her feeling left out when i am...,0
41133,"so , am i the asshole in this situation ?",0


# **Random Forest**

In [60]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
text_transformer = TfidfVectorizer(stop_words='english', lowercase=True)

In [61]:
train_x = train_sentences[["line"]]
train_y = train_sentences[["line_label"]]

dev_x = dev_sentences[["line"]]
dev_y = dev_sentences[["line_label"]]

test_x = test_sentences[["line"]]
test_y = test_sentences[["line_label"]]

In [62]:
train_x_text = text_transformer.fit_transform(train_x['line']) #train_sentences[]
dev_x_text = text_transformer.transform(dev_x['line'])
test_x_text = text_transformer.transform(test_x['line'])

In [63]:
train_x_text.shape

(228625, 50857)

In [64]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

classifier=RandomForestClassifier()
classifier.fit(train_x_text,train_y)


RandomForestClassifier()

In [66]:
y_pred2 = classifier.predict(test_x_text)


from sklearn.metrics import classification_report
# Making the Confusion Matrix
print(confusion_matrix(test_y,y_pred2)) 
print(classification_report(test_y, y_pred2)) 

[[19609 12879]
 [ 4692  3955]]
              precision    recall  f1-score   support

           0       0.81      0.60      0.69     32488
           1       0.23      0.46      0.31      8647

    accuracy                           0.57     41135
   macro avg       0.52      0.53      0.50     41135
weighted avg       0.69      0.57      0.61     41135



In [67]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 13:30:15


# **Vote for RF**

In [79]:
# get the both true and predicted labels as lists
test_true_labels = test_y
test_pred_labels = y_pred2.flatten()


#get the number of sentences of everystory in every in all datasets
#train_sentence_count = train_df['num_of_sentences'].to_numpy().tolist()
#dev_sentence_count = dev_df['num_of_sentences'].to_numpy().tolist()
test_sentence_count = test_df['num_of_sentences'] .to_numpy().tolist()



print(type(test_sentence_count))
#print(test_sentence_count)
sentence_label_copy = np.array(test_pred_labels)



<class 'list'>


In [80]:
sentence_label_copy.shape

(41135,)

In [81]:
test_list_for_vote = [] # this list contains sublists of labels of sentences for every story

for i in test_sentence_count: # loop through each sentence counts(number of sentences for each story
  sub = [] # empty list for storing all the labels that belongs to that ONE particular story
  sub.append(sentence_label_copy[:i].tolist()) 
  # select the sentences labels that belong to the corresponding story and adding to sub

  index = [x for x in range(0,i)] # the index for selection and deletion
  sentence_label_copy = np.delete(sentence_label_copy, index) # delete the existed labels

  test_list_for_vote.append(sub[0]) # add the labels(for each story) to the list for later voting



def most_common(lst):
    return max(set(lst), key=lst.count)



# double checking and vote the most commom's label for each story
test_vote_result = [] 
for i in test_list_for_vote:
  if len(i) != 0:
    test_vote_result.append(most_common(i))
  else:
    test_vote_result.append(0)

In [82]:
# a copy of label of story
tbl =  test_df['num_binarized_label'] 
test_binarized_label =[]
for i in tbl:
  test_binarized_label.append(i)



# check the length is the same or not
print(len(test_binarized_label))
print(len(test_vote_result))



# final result
test_correct = 0
for i in range(0, 2358):
  if test_binarized_label[i] == test_vote_result[i]:
    test_correct += 1
print('accuracy:' , test_correct/2358)

2358
2358
accuracy: 0.6700593723494487


In [83]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score
print(classification_report(test_binarized_label, test_vote_result))
print(confusion_matrix(test_binarized_label, test_vote_result))

              precision    recall  f1-score   support

           0       0.81      0.75      0.78      1827
           1       0.31      0.38      0.34       531

    accuracy                           0.67      2358
   macro avg       0.56      0.57      0.56      2358
weighted avg       0.70      0.67      0.68      2358

[[1377  450]
 [ 328  203]]


In [84]:
from datetime import datetime

now = datetime.now()

current_time = now.strftime("%H:%M:%S")
print("Current Time =", current_time)

Current Time = 20:08:52
